![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_Phi3.ipynb)

# Import OpenVINO Phi3 models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and importing Phi3 models from HuggingFace  for use in Spark NLP, with [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html). The focus is on converting the model to the OpenVINO format and applying precision optimizations (INT8 and INT4), to enhance the performance and efficiency on CPU platforms using [Optimum Intel](https://huggingface.co/docs/optimum/main/en/intel/inference).

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance CPU inference for models. So please make sure you have upgraded to the latest Spark NLP release.
- Model quantization is a computationally expensive process, so it is recommended to use a runtime with more than 32GB memory for exporting the quantized model from HuggingFace.
- You can import LLama models via `Phi3Model`. These models are usually under `Text Generation` category and have `Phi3` in their labels.
- Some [example models](https://huggingface.co/models?search=Phi3)

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future release, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API. It also offers the ability to perform weight compression during export.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [Phi3/Phi31.5-7B-Chat](https://huggingface.co/Phi3/Phi31.5-7B-Chat) model from HuggingFace as an example.
- In addition to `Phi3Model` we also need to save the tokenizer. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

### Option 1: Exporting to OpenVINO IR in INT8 Precision

Passing the `load_in_8bit` parameter applies 8-bit quantization on the model weights.

In [2]:
from optimum.intel.openvino import OVQuantizer, OVModelForCausalLM
from transformers import AutoTokenizer, AutoConfig


MODEL_NAME = "microsoft/Phi-3-mini-128k-instruct"
EXPORT_PATH = f"./models/int8/{MODEL_NAME}"
ASSETS_PATH = f"{EXPORT_PATH}/assets"

model = OVModelForCausalLM.from_pretrained(MODEL_NAME, export=True, load_in_8bit=True)
model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ASSETS_PATH)

config = AutoConfig.from_pretrained(MODEL_NAME)
config.save_pretrained(ASSETS_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:285: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif sliding_window is None or key_value_length < sliding_window:
/usr/local/lib/python3.11/dist-packages/transformers/models/phi3/modeling_phi3.py:805: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.shape[-1] > target_length:
/usr/local/lib/python3.11/dist-packages/transformers/models/phi3/modeling_phi

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 100% (130 / 130)            │ 100% (130 / 130)                       │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Once the model export and quantization is complete, move the model assets needed for tokenization in Spark NLP to the `assets` directory.

In [3]:
!ls -l {EXPORT_PATH}

total 3737512
drwxr-xr-x 2 root root       4096 Jun 24 08:30 assets
-rw-r--r-- 1 root root       3424 Jun 24 08:28 config.json
-rw-r--r-- 1 root root        172 Jun 24 08:30 generation_config.json
-rw-r--r-- 1 root root        609 Jun 24 08:30 openvino_config.json
-rw-r--r-- 1 root root 3824918800 Jun 24 08:30 openvino_model.bin
-rw-r--r-- 1 root root    2266797 Jun 24 08:30 openvino_model.xml


In [5]:
import json

# Load tokenizer JSON
with open(f"{ASSETS_PATH}/tokenizer.json") as f:
    tokenizer = json.load(f)
    vocab_json = tokenizer["model"]["vocab"]
    merges = tokenizer["model"]["merges"]
    special_tokens = tokenizer.get("added_tokens", [])

# Reconstruct vocab list
vocab_size = len(vocab_json) + len(special_tokens)
vocab = [""] * vocab_size
for token, idx in vocab_json.items():
    vocab[idx] = token
for token in special_tokens:
    vocab[token["id"]] = token["content"]

# Save vocab.txt
with open(f"{ASSETS_PATH}/vocab.txt", "w") as f:
    f.write("\n".join(vocab))

# Save merges.txt
with open(f"{ASSETS_PATH}/merges.txt", "w") as f:
    f.write("\n".join([" ".join(pair) for pair in merges]))

In [6]:
!ls -l {EXPORT_PATH}/assets

total 4816
-rw-r--r-- 1 root root     293 Jun 24 08:30 added_tokens.json
-rw-r--r-- 1 root root    3426 Jun 24 08:30 config.json
-rw-r--r-- 1 root root  544426 Jun 24 08:31 merges.txt
-rw-r--r-- 1 root root     569 Jun 24 08:30 special_tokens_map.json
-rw-r--r-- 1 root root    3371 Jun 24 08:30 tokenizer_config.json
-rw-r--r-- 1 root root 3620657 Jun 24 08:30 tokenizer.json
-rw-r--r-- 1 root root  499723 Jun 24 08:30 tokenizer.model
-rw-r--r-- 1 root root  243079 Jun 24 08:31 vocab.txt


### Option 2: Exporting to OpenVINO IR in INT4 Precision

Alternately, Optimum Intel also provides [4-bit weight compression](https://huggingface.co/docs/optimum/intel/optimization_ov#4-bit) with `OVWeightQuantizationConfig` class to control weight quantization parameters. The `ratio` parameter controls the ratio between 4-bit and 8-bit quantization. If set to 0.8, it means that 80% of the layers will be quantized to int4 while the remaining 20% will be quantized to int8.

In [ ]:
from optimum.intel.openvino import OVWeightQuantizationConfig, OVModelForCausalLM
from transformers import AutoTokenizer, AutoConfig

MODEL_NAME = "microsoft/Phi-3-mini-128k-instruct"
EXPORT_PATH = f"./models/int4/{MODEL_NAME}"

q_config = OVWeightQuantizationConfig(bits=4, sym=True, group_size=128, ratio=0.8)

model = OVModelForCausalLM.from_pretrained(MODEL_NAME, export=True, quantization_config=q_config)
model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(f"{EXPORT_PATH}/assets")

config = AutoConfig.from_pretrained(MODEL_NAME)
config.save_pretrained(f"{EXPORT_PATH}/assets")

Once the model export and quantization is complete, move the model assets needed for tokenization in Spark NLP to the `assets` directory.

In [ ]:
!ls -l {EXPORT_PATH}

total 2366164
drwxr-xr-x 2 root root       4096 Mar 17 19:36 assets
-rw-r--r-- 1 root root       3424 Mar 17 19:36 config.json
-rw-r--r-- 1 root root        172 Mar 17 19:36 generation_config.json
-rw-r--r-- 1 root root        640 Mar 17 19:36 openvino_config.json
-rw-r--r-- 1 root root 2420769108 Mar 17 19:36 openvino_model.bin
-rw-r--r-- 1 root root    2157175 Mar 17 19:36 openvino_model.xml


In [ ]:
import json

# Load tokenizer JSON
with open(f"{ASSETS_PATH}/tokenizer.json") as f:
    tokenizer = json.load(f)
    vocab_json = tokenizer["model"]["vocab"]
    merges = tokenizer["model"]["merges"]
    special_tokens = tokenizer.get("added_tokens", [])

# Reconstruct vocab list
vocab_size = len(vocab_json) + len(special_tokens)
vocab = [""] * vocab_size
for token, idx in vocab_json.items():
    vocab[idx] = token
for token in special_tokens:
    vocab[token["id"]] = token["content"]

# Save vocab.txt
with open(f"{ASSETS_PATH}/vocab.txt", "w") as f:
    f.write("\n".join(vocab))

# Save merges.txt
with open(f"{ASSETS_PATH}/merges.txt", "w") as f:
    f.write("\n".join([" ".join(pair) for pair in merges]))

In [ ]:
!ls -l {EXPORT_PATH}/assets

total 4044
-rw-r--r-- 1 root root     293 Mar 17 19:36 added_tokens.json
-rw-r--r-- 1 root root    3426 Mar 17 19:36 config.json
-rw-r--r-- 1 root root     569 Mar 17 19:36 special_tokens_map.json
-rw-r--r-- 1 root root    3371 Mar 17 19:36 tokenizer_config.json
-rw-r--r-- 1 root root 3620657 Mar 17 19:36 tokenizer.json
-rw-r--r-- 1 root root  499723 Mar 17 19:36 tokenizer.model


All done! the model's ready to be loaded into Spark NLP

## 2. Import and Save Llama2 in Spark NLP



- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 35.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `LLAMA2Transformer` which allows us to load the OpenVINO model.
- Most params will be set automatically. They can also be set later after loading the model in `LLAMA2Transformer` during runtime, so don't worry about setting them now.
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [9]:
from sparknlp.annotator import Phi3Transformer

Phi3 = Phi3Transformer \
    .loadSavedModel(EXPORT_PATH, spark) \
    .setMaxOutputLength(50) \
    .setDoSample(True) \
    .setInputCols(["documents"]) \
    .setOutputCol("generation")

Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
Phi3.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your OpenVINO Phi3 model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 3738564
drwxr-xr-x 3 root root       4096 Jun 24 08:37 fields
drwxr-xr-x 2 root root       4096 Jun 24 08:37 metadata
-rw-r--r-- 1 root root 3827769865 Jun 24 08:41 phi3_openvino
-rw-r--r-- 1 root root     499723 Jun 24 08:41 phi3_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Llama2 model 😊

In [13]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Phi3Transformer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

phi3 = Phi3Transformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setMaxOutputLength(50) \
    .setInputCols(["documents"]) \
    .setOutputCol("generation")

pipeline = Pipeline().setStages([
    document_assembler,
    phi3
])

test_data = spark.createDataFrame([
    [1, "<|system|> You are a helpful assistant.<|end|> <|user|> How to explain Internet for a medieval knight?<|end|> <|assistant|>"]
], ["id", "text"])

model = pipeline.fit(test_data)
results = model.transform(test_data)

results.select("generation.result").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

That's it! You can now go wild and use hundreds of Phi3 models from HuggingFace 🤗 in Spark NLP 🚀
